# Diseño de Sistemas de Adquisición y Procesamiento Masivo de Datos

## Práctica 1

### Yago Boleas y Zakaria Lasry

In [ ]:
import os
import polars as pl
from sklearn.cluster import DBSCAN, KMeans
import plotly.express as px
import plotly.graph_objs as go
from tqdm import tqdm
from pathlib import Path

from utils import clean_lidar_points, make_voxel_grid, mark_static_points, clusters_with_dbscan, compute_centroids
from utils import update_tracking_state, predict_positions, compute_velocities, match_clusters, plot_scene, create_bounding_box_lines
from utils import max_, min_


In [ ]:
data = dict(
    carretera=Path("./data/0_carretera.csv"),
    portico=Path("./data/1_portico.csv"),
    coche=Path("./data/2_coche.csv"),
    coches=Path("./data/3_coche_coche.csv"),
    coches_moto=Path("./data/4_coche_coche_moto.csv"),
    video=Path("./data/5_video_nube_de_puntos"),
)



## ANÁLISIS DE LOS DATOS

Lo primero es echar un vistazo a las distintas variables que hay en los conjuntos de datos. Usamos el primer conjunto de datos `carretera` para esto

In [ ]:
df = pl.read_csv(data["carretera"])
df = df.unique(["x", "y", "z"])
df

### Descripción de variables

- **x**: coordenada cartesiana en el eje horizontal (eje X).
- **y**: coordenada cartesiana en el eje horizontal (eje Y).
- **z**: coordenada cartesiana en el eje vertical (eje Z).
- **intensity**: intensidad de la señal reflejada que regresa al sensor. Indica cuánta luz láser es reflejada por el objeto detectado.
- **t**: marca de tiempo (*timestamp*) en la que se tomó la medición.
- **reflectivity**: reflectividad del objeto detectado. Mide la capacidad del objeto para reflejar la luz láser.
- **ring**: índice del anillo o línea de escaneo al que pertenece el punto.
- **ambient**: nivel de luz ambiental presente en el entorno. 
- **range**: distancia desde el sensor Lidar hasta el objeto detectado, medida en metros.

### Primer vistazo a los datos

En esta figura podemos ver la carretera sin ningún tipo de obtáculo por en medio. Se ha decidido también representar la variable `range`, que representa la distancia a la que se encuentra cada uno de los puntos al sensor Lidar. Se muestra también el punto (0, 0, 0) para poder apreciar con mayor claridad esta variable.

Para representar los datos también se ha realizado un pequeño procesado de los datos, compuesto por dos etapas:

1. **Eliminación de duplicados**: se eliminaron los puntos que tenían coordenadas `x`, `y` y `z` idénticas.
2. **Filtrado de puntos atípicos**: se aplicó un método estadístico basado en el *Rango Intercuartílico (IQR)* para detectar y eliminar los puntos que estaban demasiado lejos del grupo principal. Específicamente, se eliminaron todos aquellos puntos que se encontraban a una distancia mayor de 2 veces el IQR por debajo del cuartil 25 o por encima del cuartil 75 para cada eje `(x, y, z)`.

Además de la limpieza, se ha representado la variable `range`, que indica la distancia de cada punto al sensor LiDAR, utilizando una **escala de colores** ('*hot*'). Para facilitar la comprensión espacial, se ha añadido un punto de referencia en el origen `(0, 0, 0)`. Es importante notar que, para la visualización, se han ajustado los ejes para una mejor perspectiva: el *eje Z* del gráfico representa la coordenada `y` del sensor (invertida) y el *eje Y* del gráfico la coordenada `z`.  Finalmente, los rangos de los ejes se han limitado a `[-30, 30]` para centrar la vista en el área de interés.

In [ ]:
dataset = "carretera"
color_var = "range"
df = pl.read_csv(data[dataset])
df = clean_lidar_points(df, verbose=False)
fig = plot_scene(df, color_var, dataset, pov=True)

### Algoritmo de eliminación de puntos estáticos

En esta nueva escena se emplean de nuevo los datos de la carretera vacía. El objetivo de esta parte es obtener una representación de las zonas estáticas dentro de la visión del Lidar. Para ello, se implementa un método de **voxelización** para crear un modelo estático del entorno. Cuando llegue una nueva escena, se comprobará que los datos pertenezcan o no a una de las distintas regiones. En caso de pertenecer a esta zona no se tendrán en cuenta a la hora de hacer la clusterización. El proceso para obtener esa escena estática es el siguiente:

1.  **Voxelización del espacio**: se define una región de interés en el espacio tridimensional (un cubo de $60 \times 60 \times 60$ metros, de -30 a 30 en cada eje) y se divide en una rejilla de $120 \times 120 \times 120$ voxels (un voxel es el equivalente tridimensional de un píxel).
2.  **Asignación de puntos a voxels**: cada punto de la nube de puntos se asigna a un voxel específico. Esto se logra calculando las coordenadas del voxel (`vx`, `vy`, `vz`) para cada punto basándose en sus coordenadas `x`, `y` y `z` y el tamaño del voxel.
3.  **Conteo de puntos por voxel**: se cuenta el número de puntos de la nube que caen dentro de cada voxel. Este conteo se almacena en el array tridimensional `voxel`. Un voxel con un conteo mayor a cero indica que esa región del espacio está "ocupada" por un objeto estático.

Para dar una idea del resultado de la voxelización se muestra una figura tridimensional con todos los voxels hallados, donde:

-   El **tamaño** de los marcadores (cada uno de los puntos) en el gráfico está escalado en función del tamaño del voxel, lo que da una idea de la granularidad de la rejilla.
-   El **color** de cada marcador representa el **conteo de puntos** en ese voxel. Esto permite identificar áreas con mayor densidad de puntos, como las superficies de la carretera o la estructura del pórtico.
-   Para una visualización más intuitiva, los ejes del gráfico se han intercambiado (`x` $\Rightarrow$ `-vx`, `y` $\Rightarrow$ `vz`, `z` $\Rightarrow$ `-vy`) para que el pórtico se muestre en una orientación más convencional.

Este modelo de voxels sirve como una "huella" de la escena estática. En el futuro, cuando llegue una nueva nube de puntos, cualquier punto que no caiga en un voxel previamente ocupado se considera parte de un **objeto móvil**, lo que facilita su detección y aislamiento.

In [ ]:
static_pts = pl.read_csv(data["carretera"])
static_pts = static_pts.unique(subset=["x", "y", "z"])
regions = 120
voxels = make_voxel_grid(static_pts, n_regions=regions)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=-voxels["vx"],
            y=voxels["vz"],
            z=-voxels["vy"],
            mode="markers",
            marker=dict(
                size=5 * ((max_ - min_) / regions),
                color=voxels["count"],
                colorbar=dict(
                    title="count",
                    tickfont=dict(size=12),
                    title_font=dict(size=14),
                ),
                opacity=0.8,
            ),
            hoverinfo="text",
            hovertext=voxels["count"],
        )
    ],
    layout=go.Layout(
        width=800,
        height=800,
        scene=dict(
            xaxis=dict(range=[-regions, 0]),
            yaxis=dict(range=[0, regions]),
            zaxis=dict(range=[-regions, 0]),
        ),
    ),
)

fig.show()

---

## 2. Pórtico (con coche)

In [ ]:
color_var = "cluster"
dataset = "portico"

df2 = pl.read_csv(data[dataset])
df2 = clean_lidar_points(df2, verbose=False)
df2 = mark_static_points(df2, voxels, n_regions=regions)
df2 = clusters_with_dbscan(
    df2,
    ["x", "y", "z"],
)

plot_scene(df2, color_var, dataset, pov=False, opacity=1)

---

## 3. Un coche

In [ ]:
color_var = "cluster"
dataset = "coche"

df3 = pl.read_csv(data[dataset])
df3 = clean_lidar_points(df3, verbose=False)
df3 = mark_static_points(df3, voxels, n_regions=regions)
df3 = clusters_with_dbscan(
    df3,
    ["x", "y", "z"],
)
plot_scene(df3, color_var, dataset, pov=True, opacity=1)

---

## Dos coches

In [ ]:
color_var = "cluster"
dataset = "coches"

df4 = pl.read_csv(data[dataset])
df4 = clean_lidar_points(df4, verbose=False)
df4 = mark_static_points(df4, voxels, n_regions=regions)
df4 = clusters_with_dbscan(
    df4,
    ["x", "y", "z"],
)
plot_scene(df4, color_var, dataset, pov=True, opacity=1)

---

## 5. Dos coches y moto

In [ ]:
color_var = "cluster"
dataset = "coches_moto"

df5 = pl.read_csv(data[dataset])
df5 = clean_lidar_points(df5, verbose=False)
df5 = mark_static_points(df5, voxels, n_regions=regions)
df5 = clusters_with_dbscan(
    df5,
    ["x", "y", "z"],
)
plot_scene(df5, color_var, dataset, pov=True, opacity=1)

---

## Uso del algoritmo KMeans

In [ ]:
df_kmeans = df5.filter(pl.col("cluster") >= -1)

K = df5.filter(pl.col("cluster") >= 0).select(pl.col("cluster").unique().len()).item()
print(f"Número de clústeres detectados por DBSCAN: {K}")

X = df5.filter(pl.col("cluster") >= -1).select(["x", "y", "z"]).to_numpy()
kmeans = KMeans(n_clusters=K, random_state=0).fit(X)

df_kmeans = df_kmeans.with_columns(pl.Series("kmeans_cluster", kmeans.labels_))
plot_scene(df_kmeans, "kmeans_cluster", dataset, pov=True, opacity=1)


Una vez vistos estos resultados, vemos que en este caso aplicar **KMeans** carece de sentido práctico por varias razones fundamentales: En primer lugar, el algoritmo exige conocer el número de clústeres $K$ antes de ejecutarse, lo cual no es viable para el caso a evaluar. El otro gran problema se puede apreciar al mostrar los datos de forma gráfica. En caso de no limpiar con una precisión impecable los datos, el algoritmo **KMeans** si o si clasificará puntos de ruido en algún clúster, los cuales con DBSCAN son tratados como lo que son. Otro de los fundamentos para evitar este algoritmo es que no hace una separación ideal de los clústeres. En la figura superior se puede ver cómo en uno de los 3 clústeres que salieron del algoritmo hay dos vehículos, cuando el objetivo de aplicar técnicas de *clustering* en este caso es separar ambos elementos.

---

## 6. Tracking de vehículos

El **tracking** consiste en identificar y mantener la trayectoria de cada objeto móvil a lo largo del tiempo, asignándole un identificador único y monitoreando su ubicación y estado continuamente. De esta forma es posible reconstruir el movimiento de los vehículos a partir de las nubes de puntos captadas por el sensor. El algoritmo empleado para realizar el tracking sobre los vehículos consiste en lo siguiente:

1.  **Clusterización de cada fotograma**: los puntos de cada escena se agrupan mediante **DBSCAN**, usando las coordenadas `(x, y, z)` para identificar los distintos vehículos y descartar el ruido (etiquetado como `-1`).

2.  **Cálculo de centroides**: para cada cluster válido se calcula el **centro geométrico** en el plano `(x, y)`, tomando el punto medio entre los valores mínimos y máximos de cada eje. Este centro representa la posición estimada del vehículo en el fotograma. Se calcula este centro geométrico y no el centroide del cluster identificado debido a que este depende de la concentración de los puntos, por lo que podría causar inconsistencias en las zonas más cercanas al sensor por estar los puntos mucho más condensados.

3.  **Emparejamiento entre fotogramas**: los centroides del fotograma actual se comparan con las **posiciones predichas** del fotograma anterior. Se calcula la distancia entre todos los centroides y se emparejan los más cercanos si la distancia es inferior a un umbral, identificando así los mismos vehículos a lo largo del tiempo.

4.  **Asignación de identificadores de seguimiento**: cada cluster recibe un `track_id`. Si un cluster actual se empareja con uno previo, mantiene su identificador; si no, se le asigna uno nuevo. Esto asegura la continuidad del seguimiento en toda la secuencia.

5.  **Cálculo de velocidades**: una vez emparejados los clusters, se estima la velocidad de cada vehículo a partir de la diferencia entre sus posiciones consecutivas, normalizada por el intervalo temporal entre fotogramas.

6.  **Predicción de posiciones futuras**: utilizando las velocidades calculadas, se predicen las posiciones de cada vehículo en el siguiente fotograma. Estas predicciones facilitan el emparejamiento incluso ante ligeras oclusiones o movimientos rápidos.

7.  **Actualización del estado de tracking**: se registra en un DataFrame todas las variables relevantes (`cluster`, `track_id`, posición, velocidad y predicción) para cada fotograma, permitiendo reconstruir la trayectoria de todos los vehículos.

Al final del proceso, cada nube de puntos queda enriquecida con la información de cluster y seguimiento, permitiendo analizar el movimiento individual de los vehículos y su evolución temporal.


In [ ]:
lazy_dfs = [
    pl.scan_csv(os.path.join(data["video"], file)).with_columns(
        pl.lit(i).alias("frame")
    )
    for i, file in enumerate(sorted(os.listdir(data["video"])))
]
df_raw = pl.concat(lazy_dfs).collect()
df_raw = df_raw.filter(pl.col("x") < 0)
frames = df_raw["frame"].unique().sort().to_list()

In [ ]:
dbscan = DBSCAN(eps=2, min_samples=20)

processed_dfs = []
prev_centroids, prev_preds = {}, {}
next_track_id = 0
prev_id_to_track_id = {}
tracking_state = pl.DataFrame(
    schema={
        "frame": pl.Int64,
        "cluster": pl.Int64,
        "cx": pl.Float64,
        "cy": pl.Float64,
        "vx": pl.Float64,
        "vy": pl.Float64,
        "px": pl.Float64,
        "py": pl.Float64,
    }
)

for frame_id in tqdm(frames, desc="Tracking vehicles"):
    frame_df = df_raw.filter(pl.col("frame") == frame_id)
    points = frame_df.select(["x", "y", "z"]).to_numpy()

    labels = dbscan.fit_predict(points)
    centroids = compute_centroids(points, labels)

    matches = match_clusters(prev_preds, centroids, threshold=2.5)
    current_id_to_track_id = {}

    for new_id in centroids.keys():
        prev_id = matches.get(new_id)
        if prev_id is not None and prev_id in prev_id_to_track_id:
            track_id = prev_id_to_track_id[prev_id]
        else:
            track_id = next_track_id
            next_track_id += 1
        current_id_to_track_id[new_id] = track_id

    track_ids = (
        pl.Series("track_id", labels).replace(current_id_to_track_id).fill_null(-1)
    )
    processed_dfs.append(frame_df.with_columns(pl.Series("cluster", labels), track_ids))

    velocities = compute_velocities(prev_centroids, centroids, matches)
    preds = predict_positions(centroids, velocities)

    tracking_state = update_tracking_state(
        tracking_state,
        frame_id=frame_id,
        centroids=centroids,
        velocities=velocities,
        preds=preds,
    )

    prev_centroids = centroids
    prev_preds = preds
    prev_id_to_track_id = current_id_to_track_id

In [ ]:
df_clustered = pl.concat(processed_dfs)

x_min, x_max = df_clustered["x"].min(), df_clustered["x"].max()
y_min, y_max = df_clustered["y"].min(), df_clustered["y"].max()
z_min, z_max = df_clustered["z"].min(), df_clustered["z"].max()

padding_factor = 1
x_range = x_max - x_min
y_range = y_max - y_min
z_range = z_max - z_min

x_min -= x_range * padding_factor
x_max += x_range * padding_factor
y_min -= y_range * padding_factor
y_max += y_range * padding_factor
z_min -= z_range * padding_factor
z_max += z_range * padding_factor

colors = px.colors.qualitative.Plotly
animation_frames = []
for f in tqdm(frames, desc="Adding bounding boxes to the frames"):
    frame_traces = []
    current_frame_data = df_clustered.filter(pl.col("frame") == f)

    noise_points = current_frame_data.filter(pl.col("track_id") == -1)
    if not noise_points.is_empty():
        frame_traces.append(
            go.Scatter3d(
                x=noise_points["x"],
                y=noise_points["y"],
                z=noise_points["z"],
                mode="markers",
                marker=dict(size=1.5, color="grey", opacity=0.5),
                name="Noise",
            )
        )

    tracked_ids = (
        current_frame_data.filter(pl.col("track_id") >= 0)["track_id"]
        .unique()
        .sort()
        .to_list()
    )

    for track_id in tracked_ids:
        cluster_points = current_frame_data.filter(pl.col("track_id") == track_id)
        color = colors[track_id % len(colors)]

        frame_traces.append(
            go.Scatter3d(
                x=cluster_points["x"],
                y=cluster_points["y"],
                z=cluster_points["z"],
                mode="markers",
                marker=dict(size=2, color=color),
                name=f"Track ID {track_id}",
            )
        )

        min_vals = cluster_points.select(["x", "y", "z"]).min().row(0)
        max_vals = cluster_points.select(["x", "y", "z"]).max().row(0)
        x_lines, y_lines, z_lines = create_bounding_box_lines(
            min_vals[0], max_vals[0], min_vals[1], max_vals[1], min_vals[2], max_vals[2]
        )
        frame_traces.append(
            go.Scatter3d(
                x=x_lines,
                y=y_lines,
                z=z_lines,
                mode="lines",
                line=dict(color=color, width=2.5),
                name=f"Box {track_id}",
            )
        )

    animation_frames.append(go.Frame(data=frame_traces, name=str(f)))

fig = go.Figure(
    data=animation_frames[0].data if animation_frames else [],
    layout=go.Layout(
        width=900,
        height=700,
        scene=dict(
            # Use the specific ranges for each axis
            xaxis=dict(title="X", range=[x_min, x_max]),
            yaxis=dict(title="Y", range=[y_min, y_max]),
            zaxis=dict(title="Z", range=[z_min, z_max]),
            # Change aspectmode to allow axes to fit their data
            aspectmode="data",
        ),
        legend=dict(itemsizing="constant", font=dict(size=10)),
    ),
    frames=animation_frames,
)

fig.update_layout(
    title=dict(
        text="Lidar Point Cloud with Persistent Cluster Tracking",
        x=0.5,
        y=0.95,
        xanchor="center",
        yanchor="top",
        font=dict(size=24),
    ),
    font=dict(family="Arial, monospace", size=12, color="Black"),
    sliders=[
        dict(
            steps=[
                dict(
                    args=[
                        [str(f)],
                        dict(frame={"duration": 200, "redraw": True}, mode="immediate"),
                    ],
                    label=str(f),
                    method="animate",
                )
                for f in frames
            ],
            transition={"duration": 0},
            x=0.1,
            y=0,
            len=0.9,
        )
    ],
)

fig.show()
#fig.write_html("animacion_lidar_3d.html")
